# Surface Water Extraction Methodology

This notebook defines the methodological approach used to extract surface water extent
from satellite imagery.

Rather than assuming a single optimal method, the focus is placed on how methodological
decisions influence results and their interpretation within a decision-making context.


## 1. Introduction

Surface water is not directly observed by satellite sensors. Instead, it is inferred
from spectral responses that are influenced by sensor characteristics, atmospheric
conditions, and surface properties.

As a result, surface water extraction is inherently a methodological decision rather
than a purely observational process.


## 2. Decision context

Estimates of surface water extent derived from satellite imagery are frequently used to
support environmental monitoring, planning, and management decisions.

These estimates depend on a sequence of methodological choices, including index
selection, threshold definition, and temporal aggregation.

Understanding the impact of these choices is essential for responsible interpretation
and use of the results.


## 3. Imports and initialization


In [ ]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt

ee.Initialize()

## 4. Data overview and preprocessing

This notebook assumes that data access, quality filtering, and region-of-interest
definition have been addressed previously.

Here, the cleaned Landsat collection and the final region of interest are loaded for
methodological analysis.


In [ ]:
roi = ee.FeatureCollection('users/your_user/fuquene_roi')

landsat = (
    ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
    .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2'))
    .merge(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2'))
    .filterBounds(roi)
    .filterDate('1985-01-01', '2024-12-31')
)

## 5. Cloud masking

Cloud contamination is a major source of uncertainty in optical satellite imagery.

A simple quality-based cloud mask is applied to reduce noise while preserving temporal
coverage.


In [ ]:
def mask_clouds(image):
    qa = image.select('QA_PIXEL')
    cloud_mask = qa.bitwiseAnd(1 << 3).eq(0)
    return image.updateMask(cloud_mask)

landsat_clean = landsat.map(mask_clouds)

## 6. Spectral index for water detection

Surface water is inferred using spectral indices designed to enhance the contrast
between water and non-water surfaces.

In this project, the Normalized Difference Water Index (NDWI) is used due to its
simplicity, interpretability, and widespread adoption.


In [ ]:
def add_ndwi(image):
    ndwi = image.normalizedDifference(['SR_B2', 'SR_B4']).rename('NDWI')
    return image.addBands(ndwi)

landsat_ndwi = landsat_clean.map(add_ndwi)

## 7. Threshold-based water classification

Transforming continuous NDWI values into a binary water / non-water classification
requires the selection of a threshold.

Threshold selection is a methodological decision that directly influences the estimated
surface water extent.


In [ ]:
def water_mask(image, threshold):
    return image.select('NDWI').gt(threshold)

## 8. Example visualization and sensitivity

To illustrate the impact of threshold selection, water masks are generated for a single
representative year using different threshold values.

This comparison highlights how small methodological changes can produce substantially
different spatial outcomes.


In [ ]:
example_image = landsat_ndwi.filterDate('2000-01-01', '2000-12-31').median()

water_01 = water_mask(example_image, 0.1)
water_02 = water_mask(example_image, 0.2)

Map = geemap.Map(center=[5.45, -73.75], zoom=11)
Map.addLayer(example_image.select('NDWI'), {'min': -1, 'max': 1}, 'NDWI')
Map.addLayer(water_01, {'palette': ['blue']}, 'Water (NDWI > 0.1)')
Map.addLayer(water_02, {'palette': ['cyan']}, 'Water (NDWI > 0.2)')
Map

## 9. Methodological sensitivity and uncertainty

The comparison demonstrates that there is no single objectively correct threshold for
water classification.

Estimated surface water extent depends on methodological assumptions, sensor
characteristics, and spatial resolution.

These factors must be considered when interpreting results and using them to inform
decisions.


## 10. Output of this notebook

This notebook produces a reproducible methodology for surface water extraction,
including:

- A defined spectral index
- An explicit threshold-based classification approach
- Clear documentation of methodological assumptions

These outputs serve as inputs for temporal analysis in the following notebook.
